In [ ]:
import os
from PIL import Image
import random
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
import tensorflow as tf; 
print(tf.version)

In [ ]:
input_width = 28
input_height = 28
output_classes = 11

In [ ]:
%matplotlib inline

def read_image(filePath):
    image = Image.open(filePath)

    # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.resize
    image = image.resize((input_width, input_height), Image.LANCZOS)

    # https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.convert
    image = image.convert("L")

    image = np.asarray(image)

    return np.asarray(image).astype(np.float32) / 255.

def read_images(path):
    files = os.listdir(path)    
    files = [file for file in files if file[-4:] == ".png"]
    random.shuffle(files)
    images = []
    for file in files:
        try:
            images.append(read_image(os.path.join(path, file)))
        except OSError:
            pass
    
    return images

test_samples_per_digit = 5
x_train_arr = []
y_train_arr = []
x_test_arr = []
y_test_arr = []
for i in range(output_classes):
    images = read_images("images/labeled/" + str(i))
    print("Digit: " + str(i) + "; Images: " + str(len(images)))
    #plt.imshow(images[0])
    #plt.show()
    idx = 0
    for image in images:
        if idx < test_samples_per_digit:
            x_test_arr.append(image)
            y_test_arr.append(i)  
        else:
            x_train_arr.append(image)
            y_train_arr.append(i)
        idx = idx + 1
    
# convert to numpy types
x_train = np.asarray(x_train_arr)#.astype(np.float32) / 255.
y_train = to_categorical(y_train_arr)
x_test = []
y_test = []
if test_samples_per_digit > 0:
    x_test = np.asarray(x_test_arr)#.astype(np.float32) / 255.
    y_test = to_categorical(y_test_arr)
#print("x_train: " + str(x_train))
#print("y_train: " + str(y_train))
#print("##########################")
#print("x_test: " + str(x_test))
#print("y_test: " + str(y_test))

In [ ]:
def createModel():
    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation = 'relu', input_shape = (input_height, input_width, 1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.5))
    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.5))
    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_classes, activation = 'softmax'))

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])
    return model

In [ ]:
model = createModel()
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(width_shift_range=3, height_shift_range=3, zoom_range=0.1, horizontal_flip=False)

if False:
    model.fit(
        x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1),
        y_train,
        epochs=2000,
        batch_size=256)
else:
    model.fit_generator(gen.flow(
        x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1), 
        y_train, batch_size=128, shuffle=True), epochs=100, workers=24, steps_per_epoch=256)

In [ ]:
def predict(img):
    result = model.predict(img.reshape(1, img.shape[0], img.shape[1], 1))
    pred = np.argmax(result, axis = 1)[0]
    convidence = result[0][pred]
    return (pred, convidence)

def predict_test(idx):
    result = predict(x_test[idx])
    pred = result[0]
    convidence = result[1]
    expect = y_test_arr[idx]
    correct = expect == pred    
    print("correct: " + str(correct) + 
          "; expected: " + str(expect) + 
          "; predicted: " + str(pred) + 
          "; confidence: " + str(round(convidence, 2)))
    if not correct or convidence < 0.7:
        plt.imshow(x_test[idx])
        plt.show()
for i in range(len(x_test_arr)):
    predict_test(i)

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
filename = "ocr_model_" + str(input_width) + "x" + str(input_height) + "_c" + str(output_classes)
print("writing " + filename + ".tflite")
open(filename + ".tflite", "wb").write(tflite_model)
print("Now you have to call the following in git bash: xxd -i " + filename + ".tflite > " + filename + ".c")